# __Capstone project: The Battle of Neighborhoods - Week 1__

## __Table of contents__

* Introduction/Business Problem
* Data
* Methodology
* Results and Discussion
* Conclusion

### Introduction/Business Problem

In this project I would like to make a description of the activities of the most important coastal cities of southern Italy and how they affect their economy.  
For this purpose I will consider the most important provinces and the most famous tourist resorts of Campania, Puglia, Calabria and the two big islands Sicily and Sardinia.

### Data description

Regarding the data on the activities of the cities I will use the API of Foursquare within the geocoder Nominatim.While regarding the economic aspect the data will be taken from the website of 'Sole 24 ore', the most important Italian economic newspaper, year 2016. This is the reference link: https://lab24.ilsole24ore.com/mappaRedditi/redditiTabelle.html

### Metodhology

After having loaded the libraries that we will use for the project and having defined the various functions to speed up the procedures, we use the FourSquare query that will give us the data of all the various activities near the various cities, which are the Sicilian Palermo, Catania, Siracusa, Ragusa, Trapani, Agrigento and Messina, the Sardinian Cagliari and Sassari, the Campanian Naples and Salerno and the Apulian Bari, Lecce, Brindisi and Taranto. We will also add the data of the annual income of the various cities (which in Italy is called 'PIL')

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from pandas.io.json import json_normalize
import requests
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print("Intallation done")

Solving environment: done

# All requested packages already installed.

Intallation done


In [7]:
geolocator = Nominatim(user_agent="myexplorer")
CLIENT_ID = '4JW2V1AXWJXREM12VKKJCM4B50FZ1I4TYRS2URMC1WMZRTNM' # your Foursquare ID
CLIENT_SECRET = 'MIJZAJNLVUZ0GB1DWIEPGAK4XXJVQFFZFOUT2EN1DHA4E2D4' # your Foursquare Secret
ACCESS_TOKEN = 'VP2X2PUFBQYAVKTTZUEIUCB3QYSNE5BI3ZOZSGOXU2XQZSTE' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 110
def getCoordinatesOf(address):
    location = geolocator.geocode(address)
    if location == None:
        print('{} not exist'.format(address))
        return location
    coord = [location.latitude, location.longitude]
    print('Coordinates of {} are {}'.format(location, coord))
    return [location.latitude, location.longitude]

def getAddressOf(coord):
    location = geolocator.reverse(coord)
    if location == None:
        print('Invalid coordinates')
        return location
    address = location.address
    print(address)
    return(address)

def getUrlForCategory(coord,venueCat,radius):
    return 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, coord[0], coord[1], ACCESS_TOKEN, VERSION, venue, radius, LIMIT)


def getUrlForTrendingVenues(coord):
    return 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, coord[0], coord[1], VERSION)

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getDataframeFromItems(items, type):
    #print('founded {} items'.format(len[items]))
    dataframe = pd.json_normalize(items) # flatten JSON
    # filter columns
    if type == 0:
        filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
    else:
        filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    # filter the category for each row
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    # clean columns
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
    dataframe_filtered.drop(['labeledLatLngs', 'id'],1 , inplace=True)
    if type==1:
        dataframe_filtered.drop(df.columns[1] ,1 , inplace=True)
    return dataframe_filtered

def explore(address, radius=200, venueCat=None, trending = False):
    coord = getCoordinatesOf(address)
    if venueCat==None:
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, coord[0], coord[1], VERSION, radius, LIMIT)
        results = requests.get(url).json()
        items = results['response']['groups'][0]['items']
        type = 0
    else:
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, coord[0], coord[1], ACCESS_TOKEN, VERSION, venueCat, radius, LIMIT)
        results = requests.get(url).json()
        items = results['response']['venues']
        type = 1
    df = getDataframeFromItems(items, type)
    return df

def get_address_venue_perc(address, radius):
    address_df = explore(address, radius).loc[:, ['cc', 'categories', 'distance']]
    address_onehot=pd.get_dummies(address_df[['categories']])
    address_onehot.columns = [col.split("_")[-1] for col in address_onehot.columns]
    address_onehot['City'] = address_df['cc']
    address_onehot_columns = [address_onehot.columns[-1]] + list(address_onehot.columns[:-1])
    address_onehot=address_onehot[address_onehot_columns]
    return address_onehot

def get_venues(address, radius):
    df=explore(address, radius)
    df_oneHot=pd.get_dummies(df[['categories']])
    df_oneHot.columns = [col.split("_")[-1] for col in df_oneHot.columns]
    df_oneHot['Loc']=address
    columns = [df_oneHot.columns[-1]]+list(df_oneHot.columns[:-1])
    df_oneHot=df_oneHot[columns]
    return df_oneHot


In [18]:
rg_df = get_venues('Ragusa', 20000)
rg_df['PIL']=17925
sr_df = get_venues('Siracusa', 40000)
sr_df['PIL']=20395
ct_df = get_venues('Catania', 20000)
ct_df['PIL']=20179
me_df = get_venues('Messina', 10000)
me_df['PIL']=21534
pa_df = get_venues('Palermo', 30000)
pa_df['PIL']=22264
tp_df = get_venues('Trapani', 30000)
tp_df['PIL']=18318
ag_df = get_venues('Agrigento', 40000)
ag_df['PIL']=20881
sic_df=pd.concat([rg_df,sr_df,ct_df,me_df,pa_df,tp_df,ag_df]).reset_index()
cg_df=get_venues('Cagliari',20000)
cg_df['PIL']=25681
ss_df=get_venues('Sassari',20000)
ss_df['PIL']=22165
rc_df=get_venues('Reggio Calabria',20000)
rc_df['PIL']=20079
cz_df=get_venues('Catanzaro',20000)
cz_df['PIL']=21487
cs_df=get_venues('Cosenza',30000)
cs_df['PIL']=21131
sa_df=get_venues('Salerno',30000)
sa_df['PIL']=23888
na_df=get_venues('Napoli',20000)
na_df['PIL']=22434
ba_df=get_venues('Bari',20000)
ba_df['PIL']=22947
le_df=get_venues('Lecce',20000)
le_df['PIL']=23420
br_df=get_venues('Brindisi',20000)
br_df['PIL']=19816
ta_df=get_venues('Taranto',20000)
ta_df['PIL']=21058
or_df=pd.concat([sic_df, rc_df,cz_df,cs_df,sa_df,na_df,cg_df,ss_df,ba_df,le_df,br_df,ta_df]).reset_index()
or_df.drop(['index'],1,inplace=True)
or_df=or_df.fillna(0)
col_ind=or_df.columns.get_loc('PIL')
columns = [or_df.columns[col_ind]] + list(or_df.columns[1:col_ind]) + list(or_df.columns[col_ind+1:])
or_df = or_df[columns]
or_df

Coordinates of Ragusa, Sicilia, Italia are [36.9219828, 14.7213455]
Coordinates of Siracusa, Sicilia, Italia are [37.0646139, 15.2907196]
Coordinates of Catania, Sicilia, Italia are [37.5022355, 15.08738]
Coordinates of Messina, Sicilia, Italia are [38.1937571, 15.5542082]
Coordinates of Palermo, Sicilia, Italia are [38.1112268, 13.3524434]
Coordinates of Trapani, Sicilia, 91100, Italia are [38.0174321, 12.515992]
Coordinates of Agrigento, Sicilia, 92100, Italia are [37.3122991, 13.57465]
Coordinates of Cagliari - Casteddu, Cagliari, Sardegna, Italia are [39.2169525, 9.1128513]
Coordinates of Sassari, Sardegna, Italia are [40.777800400000004, 8.921996955790174]
Coordinates of Reggio di Calabria, Calabria, Italia are [38.1035389, 15.6397556]
Coordinates of Catanzaro, Calabria, Italia are [38.82996034999999, 16.43155687627833]
Coordinates of Cosenza, Calabria, Italia are [39.596685300000004, 16.333055599019605]
Coordinates of Salerno, Campania, Italia are [40.419441649999996, 15.31075623

,PIL,Loc,Airport,Bar,Beach,Bed & Breakfast,Cafeteria,Café,Castle,Chocolate Shop,...,Gym,Mobile Phone Shop,Sports Bar,Hotel Bar,Performing Arts Venue,Women's Store,Irish Pub,Beer Bar,Convenience Store,Shoe Store
0,17925,Ragusa,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17925,Ragusa,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17925,Ragusa,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17925,Ragusa,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17925,Ragusa,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,21058,Taranto,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1364,21058,Taranto,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1365,21058,Taranto,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1366,21058,Taranto,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


At this point we obtain the table that gives us for each city the average of the various activities

In [19]:
or_df_grouped=or_df.groupby('Loc').mean().reset_index()
or_df_grouped

,Loc,PIL,Airport,Bar,Beach,Bed & Breakfast,Cafeteria,Café,Castle,Chocolate Shop,...,Gym,Mobile Phone Shop,Sports Bar,Hotel Bar,Performing Arts Venue,Women's Store,Irish Pub,Beer Bar,Convenience Store,Shoe Store
0,Agrigento,20881,0.000000,0.000000,0.159420,0.000000,0.000000,0.086957,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bari,22947,0.000000,0.030000,0.020000,0.000000,0.000000,0.080000,0.000000,0.000000,...,0.01,0.01,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Brindisi,19816,0.000000,0.013699,0.082192,0.000000,0.000000,0.013699,0.013699,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.013699,0.000000,0.000000,0.000000
3,Cagliari,25681,0.000000,0.000000,0.056818,0.000000,0.011364,0.045455,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Catania,20179,0.000000,0.000000,0.020000,0.030000,0.000000,0.040000,0.020000,0.010000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Catanzaro,21487,0.000000,0.022222,0.122222,0.000000,0.000000,0.088889,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Cosenza,21131,0.000000,0.000000,0.142857,0.020408,0.020408,0.102041,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Lecce,23420,0.000000,0.021505,0.075269,0.000000,0.000000,0.086022,0.010753,0.000000,...,0.00,0.00,0.00,0.010753,0.010753,0.010753,0.000000,0.000000,0.000000,0.000000
8,Messina,21534,0.000000,0.000000,0.016393,0.000000,0.000000,0.180328,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Napoli,22434,0.000000,0.000000,0.020000,0.010000,0.000000,0.060000,0.030000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Results

We are interested in knowing which activity has the greatest impact on income, so we use the corr() function that Phyton gives us to measure the correlation between PIL and the other components

In [29]:
or_df_grouped.corr()['PIL'].sort_values().head(10)

Comfort Food Restaurant   -0.628433
Deli / Bodega             -0.612401
City                      -0.544690
Diner                     -0.484220
Sicilian Restaurant       -0.483076
Vineyard                  -0.456317
Hobby Shop                -0.456317
Airport                   -0.456317
Chocolate Shop            -0.455104
Neighborhood              -0.455104
Name: PIL, dtype: float64

This data concerns the activities that negatively affect income, strangely enough we notice that among the 10 activities with greater negative influence there are those related to food at various levels.
Let's see now which are the 10 activities with the greatest positive influence on income

In [31]:
or_df_grouped.corr()['PIL'].sort_values(ascending=False).head(11)

PIL                     1.000000
Coffee Shop             0.680327
Dive Bar                0.565296
Rock Climbing Spot      0.555653
Beach Bar               0.555653
College Soccer Field    0.555653
Flower Shop             0.555653
Noodle House            0.555653
Art Gallery             0.555653
Garden                  0.508428
Tea Room                0.498639
Name: PIL, dtype: float64

We note that activities such as bars and tea rooms and activities related to floriculture have a positive influence on income.

### Conclusion

It is clear that 'light' activities with little investment but of good interest have a positive impact on income, unlike activities that are too costly.